In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import cv2
import os
import PIL
import pathlib
import splitfolders

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.applications.efficientnet import EfficientNetB3
from keras.applications.inception_v3 import InceptionV3

In [3]:
batch_size = 128
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)

In [4]:
train_datagen = ImageDataGenerator(
    shear_range = 0.2, 
    zoom_range = 0.2)

val_datagen = ImageDataGenerator()

train_ds = train_datagen.flow_from_directory(
    'imgs_splitted/train',
    target_size = (img_height, img_width),
    batch_size = batch_size,
    seed = 123,
    class_mode='categorical')

val_ds = val_datagen.flow_from_directory(
    'imgs_splitted/val',
    target_size = (img_height, img_width),
    batch_size = batch_size,
    seed = 123,
    class_mode='categorical',
    shuffle=False)

test_ds = val_datagen.flow_from_directory(
    'imgs_splitted/test',
    target_size = (img_height, img_width),
    batch_size = batch_size,
    seed = 123,
    class_mode='categorical',
    shuffle=False)


Found 14681 images belonging to 8 classes.
Found 1833 images belonging to 8 classes.
Found 1842 images belonging to 8 classes.


In [4]:
inputs = tf.keras.Input(input_shape)
efficientnet = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top=False,
    weights='imagenet', input_tensor=inputs )
efficientnet.trainable = False
x = efficientnet(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(8, activation='softmax')(x)
model_efficientnet = tf.keras.Model(inputs, x)

In [5]:
model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_efficientnet.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 7, 7, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                             

In [25]:
checkpointer = ModelCheckpoint(filepath='new_models/model_efficientnetv2.hdf5', 
                               monitor='val_accuracy', mode='max',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.0001)
callbacks=[early_stopping, reduce_lr, checkpointer]

In [33]:
history1 = model_efficientnet.fit(train_ds, epochs = 100, validation_data = val_ds, callbacks=callbacks)

Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.2732 - accuracy: 0.9108
Epoch 1: val_accuracy improved from -inf to 0.92253, saving model to new_models\model_efficientnetv2.hdf5
115/115 [==============================] - 125s 1s/step - loss: 0.2732 - accuracy: 0.9108 - val_loss: 0.4244 - val_accuracy: 0.9225 - lr: 0.0010
Epoch 2/100
115/115 [==============================] - ETA: 0s - loss: 0.0850 - accuracy: 0.9738
Epoch 2: val_accuracy improved from 0.92253 to 0.96072, saving model to new_models\model_efficientnetv2.hdf5
115/115 [==============================] - 119s 1s/step - loss: 0.0850 - accuracy: 0.9738 - val_loss: 0.1501 - val_accuracy: 0.9607 - lr: 0.0010
Epoch 3/100
115/115 [==============================] - ETA: 0s - loss: 0.0777 - accuracy: 0.9766
Epoch 3: val_accuracy improved from 0.96072 to 0.96618, saving model to new_models\model_efficientnetv2.hdf5
115/115 [==============================] - 122s 1s/step - loss: 0.0777 - accuracy: 0.9766 - val

In [26]:
model_efficientnet.load_weights('new_models/model_efficientnetv2.hdf5')

In [27]:
efficientnet.trainable = True

model_efficientnet.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
history2 = model_efficientnet.fit(train_ds, epochs = 100, validation_data = val_ds, callbacks=callbacks)

Epoch 1/100
230/230 [==============================] - ETA: 0s - loss: 0.0243 - accuracy: 0.9932
Epoch 1: val_accuracy improved from -inf to 0.99236, saving model to new_models\model_efficientnetv2.hdf5
230/230 [==============================] - 132s 544ms/step - loss: 0.0243 - accuracy: 0.9932 - val_loss: 0.0324 - val_accuracy: 0.9924 - lr: 1.0000e-05
Epoch 2/100
230/230 [==============================] - ETA: 0s - loss: 0.0113 - accuracy: 0.9962
Epoch 2: val_accuracy did not improve from 0.99236
230/230 [==============================] - 121s 527ms/step - loss: 0.0113 - accuracy: 0.9962 - val_loss: 0.0291 - val_accuracy: 0.9918 - lr: 1.0000e-05
Epoch 3/100
230/230 [==============================] - ETA: 0s - loss: 0.0112 - accuracy: 0.9965
Epoch 3: val_accuracy improved from 0.99236 to 0.99345, saving model to new_models\model_efficientnetv2.hdf5
230/230 [==============================] - 121s 523ms/step - loss: 0.0112 - accuracy: 0.9965 - val_loss: 0.0200 - val_accuracy: 0.9935 - lr

In [29]:
model_efficientnet.load_weights('new_models/model_efficientnetv2.hdf5')

In [33]:
score1 = model_efficientnet.evaluate(val_ds, verbose=1)

29/29 [==============================] - 5s 171ms/step - loss: 0.0130 - accuracy: 0.9973


In [15]:
inputs = tf.keras.Input(input_shape)
efficientnet = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights='imagenet', input_tensor=inputs )
efficientnet.trainable = False
x = efficientnet(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(8, activation='softmax')(x)
model_efficientnet = tf.keras.Model(inputs, x)

In [16]:
model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_efficientnet.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_2 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_4 (Dense)             (None, 1024)              1311

In [8]:
checkpointer = ModelCheckpoint(filepath='new_models/model_efficientnetb0.hdf5', 
                               monitor='val_accuracy', mode='max',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.0001)
callbacks=[early_stopping, reduce_lr, checkpointer]

In [9]:
history3 = model_efficientnet.fit(train_ds, epochs = 100, validation_data = val_ds, callbacks=callbacks)

Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.2989 - accuracy: 0.9014
Epoch 1: val_accuracy improved from -inf to 0.94435, saving model to new_models\model_efficientnetb0.hdf5
115/115 [==============================] - 171s 1s/step - loss: 0.2989 - accuracy: 0.9014 - val_loss: 0.3790 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 2/100
115/115 [==============================] - ETA: 0s - loss: 0.1017 - accuracy: 0.9667
Epoch 2: val_accuracy improved from 0.94435 to 0.96236, saving model to new_models\model_efficientnetb0.hdf5
115/115 [==============================] - 122s 1s/step - loss: 0.1017 - accuracy: 0.9667 - val_loss: 0.1569 - val_accuracy: 0.9624 - lr: 0.0010
Epoch 3/100
115/115 [==============================] - ETA: 0s - loss: 0.0675 - accuracy: 0.9789
Epoch 3: val_accuracy improved from 0.96236 to 0.98309, saving model to new_models\model_efficientnetb0.hdf5
115/115 [==============================] - 121s 1s/step - loss: 0.0675 - accuracy: 0.9789 - val

In [10]:
model_efficientnet.load_weights('new_models/model_efficientnetb0.hdf5')

In [11]:
for layer in efficientnet.layers[-20:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

model_efficientnet.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history4 = model_efficientnet.fit(train_ds, epochs = 100, validation_data = val_ds, callbacks=callbacks)

Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.0106 - accuracy: 0.9965
Epoch 1: val_accuracy did not improve from 0.99291
115/115 [==============================] - 122s 1s/step - loss: 0.0106 - accuracy: 0.9965 - val_loss: 0.0268 - val_accuracy: 0.9918 - lr: 1.0000e-05
Epoch 2/100
115/115 [==============================] - ETA: 0s - loss: 0.0125 - accuracy: 0.9963
Epoch 2: val_accuracy did not improve from 0.99291
115/115 [==============================] - 120s 1s/step - loss: 0.0125 - accuracy: 0.9963 - val_loss: 0.0265 - val_accuracy: 0.9918 - lr: 1.0000e-05
Epoch 3/100
115/115 [==============================] - ETA: 0s - loss: 0.0111 - accuracy: 0.9963
Epoch 3: val_accuracy did not improve from 0.99291
115/115 [==============================] - 119s 1s/step - loss: 0.0111 - accuracy: 0.9963 - val_loss: 0.0261 - val_accuracy: 0.9918 - lr: 1.0000e-05
Epoch 4/100
115/115 [==============================] - ETA: 0s - loss: 0.0104 - accuracy: 0.9966
Epoch 4: val

In [17]:
model_efficientnet.load_weights('new_models/model_efficientnetb0.hdf5')

In [18]:
score2 = model_efficientnet.evaluate(val_ds, verbose=1)
print(f'EfficientNet Loss: {score2[0]}, Accuracy: {score2[1]*100}')

15/15 [==============================] - 6s 317ms/step - loss: 0.0280 - accuracy: 0.9929
EfficientNet Loss: 0.027964744716882706, Accuracy: 99.29078221321106
